In [1]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu
import cv2
import sys

In [2]:
def getColorExact(localExtrema, ntscIm):
    [n,m,d] = ntscIm.shape
    imgSize = n*m
    nI = np.zeros(ntscIm.shape, dtype=ntscIm.dtype)
    nI[...,0] = ntscIm[...,0]
    indsM = np.arange(imgSize).reshape(n, m)

    wd = 1
    length = 0
    consts_len = 0
    col_inds = np.zeros(imgSize*(2*wd+1)**2, dtype=int)
    row_inds = np.zeros(imgSize*(2*wd+1)**2, dtype=int)
    vals = np.zeros(imgSize*(2*wd+1)**2)
    gvals = np.zeros((2*wd+1)**2)

    for i in range(n):
        for j in range(m):
            if not localExtrema[i,j]:
                tlen = 0
                for ii in range(max(0,i-wd),min(i+wd+1,n)):
                    for jj in range(max(0,j-wd),min(j+wd+1,m)):
                        if(ii != i) or (jj != j):
                            row_inds[length] = consts_len
                            col_inds[length] = indsM[ii,jj]
                            gvals[tlen] = ntscIm[ii,jj,0]
                            length  = length +1
                            tlen = tlen+1
                t_val = ntscIm[i,j,0] # center pixel Y value
                gvals[tlen] = t_val
                c_var = np.mean((gvals[:tlen+1] - np.mean(gvals[:tlen+1]) )**2)
                csig=c_var*0.6
                mgv = min((gvals[:tlen]-t_val)**2)
                if csig < (-mgv/np.log(0.01)):
                    csig = -mgv/np.log(0.01)
                if csig < 0.000002:
                    csig = 0.000002
                gvals[:tlen] = np.exp(-(gvals[:tlen]-t_val)**2/csig)
                gvals[:tlen] = gvals[:tlen]/sum(gvals[:tlen])
                vals[length-tlen:length] = -gvals[:tlen]

            row_inds[length] = consts_len
            col_inds[length] = indsM[i,j]
            vals[length]=1
            length = length+1    
            consts_len = consts_len+1

    vals = vals[:length]
    col_inds = col_inds[:length]
    row_inds = row_inds[:length]

    A_csc = csc_matrix((vals, (row_inds, col_inds)), shape=(consts_len, imgSize))
    LU = splu(A_csc)
    b = np.zeros(A_csc.shape[0],dtype=ntscIm.dtype )
    for dim in range(1,d):
        curIm = ntscIm[:,:,dim]
        b[indsM[localExtrema != 0]] = curIm[localExtrema]
        new_vals = LU.solve(b)
        nI[...,dim] = new_vals.reshape((n,m))
        
    return nI

In [3]:
def EdgePreservingSmooth(I,k=3):
    """
    Implement "Edge-preserving Multiscale Image Decomposition based on Local Extrema"

    Parameters
    -----------
    I: input image( BGR image or grayscale image )
    k: kernel size, default = 3

    Returns
    -----------
    M: smoothed image( BGR image or grayscale image )
    localMax: local maxima extrema( boolean matrix )
    localMin: local minima extrema( boolean matrix )
    MaxEnvelope: extremal envelopes of maxima extrema( Y+ extremal envelopes at each BGR channel )
    MinEnvelope: extermal envelope of minima extrema( Y+ extremal envelopes at each BGR channel )

    """

    # wd: half width of kernel size
    wd = k//2

    if I.ndim == 3:
        channel = I.shape[2]
        YUV = cv2.cvtColor(I,cv2.COLOR_BGR2YUV)
        Y = np.double(YUV[:,:,0])/255
        image = np.double(I)/255
        #cv2.imshow("Y",Y)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

    else:
        channel = 1
        Y = np.double(I)/255
    print("Identifcation of local minima and local maxima of I")
    height,width = Y.shape
    localMax = np.zeros( Y.shape, dtype=bool)
    localMin = np.zeros( Y.shape, dtype=bool)
    for i in range(height):
        for j in range(width):
            center = Y[i,j]

            ii_start = max(0,i-wd)
            ii_end = min(i+wd+1,height)
            jj_start = max(0,j-wd)
            jj_end = min(j+wd+1,width)
            cover = Y[ii_start:ii_end,jj_start:jj_end]

            maxcount = np.sum(cover > center)
            mincount = np.sum(center > cover)
            if maxcount <= k-1:
                localMax[i,j] = True
            if mincount <= k-1:
                localMin[i,j] = True

    print("Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively")
    Y_BGR = np.zeros((height,width,4))
    Y_BGR[...,0] = Y;
    for i in range(channel):
        Y_BGR[...,i+1] = image[...,i]

    MaxEnvelope = getColorExact(localMax, Y_BGR)
    MinEnvelope = getColorExact(localMin, Y_BGR)

    print("Computation of the smoothed mean M as the average of the extremal envelopes")
    M = (MaxEnvelope[:,:,1:(channel+1)] + MinEnvelope[:,:,1:(channel+1)])/2;
    M = (M*255).astype(np.uint8)
    #cv2.imshow("M",M)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return M, localMax, localMin, MaxEnvelope, MinEnvelope

In [4]:
def normalize(img):
    '''
    img: float image
    result: 0~255 np.uint8
    '''
    array_max = np.max(img)
    array_min = np.min(img)
    array_range = array_max-array_min
    result = img.copy()
    result = ((result-array_min)/array_range)*255
    return result.astype(np.uint8)

In [ ]:
#I1 = cv2.imread('Original.jpg')
#M1, localmax, localmin, maxenvelope, minenvelope = EdgePreservingSmooth(I1)

In [ ]:
if __name__ == '__main__':

    if len(sys.argv) < 4:
        print('Usage:', sys.argv[0], '<ImagePath>', '<KernelSize>', '<Iteration>')
        sys.exit(1)

    imagepath = sys.argv[1]
    kernelsize = int(sys.argv[2])
    iteration = int(sys.argv[3])

    I = cv2.imread(imagepath)
    M= I.copy()
    for i in range(iteration):
        print('Iteration: ', str(i+1))
        M,localmax, localmin, maxenvelope, minenvelope = EdgePreservingSmooth(M,kernelsize)
        kernelsize += 4
        print('')

    I_YUV = cv2.cvtColor(I,cv2.COLOR_BGR2YUV)
    M_YUV = cv2.cvtColor(M,cv2.COLOR_BGR2YUV)
    D = I_YUV[:,:,0]-M_YUV[:,:,0]

    # Make the grey scale image have three channels
    grey_3_channel = cv2.cvtColor(D, cv2.COLOR_GRAY2BGR)
    numpy_horizontal = np.hstack(( I, M, grey_3_channel))
    cv2.imshow('Edge-preserving Smooth Result', numpy_horizontal)
    cv2.waitKey()

# Resize images

In [15]:
"""
pascale.jpg --- enhancement
stacey.jpg --- noise
jason.jpg --- noise
michael.jpg --- detail 3 iteration
ricardo.jpg --- detail 3 iteration
daniel.jpg --- enhancement
"""
imagepath = "daniel.jpg"
img = cv2.imread(imagepath)
#cv2.imshow('Image', img)
#cv2.waitKey()
print(img.shape)
res = cv2.resize(img,(0,0),fx=0.8,fy=0.8, interpolation = cv2.INTER_AREA)
print(res.shape)
cv2.imshow('Resized Image', res)
cv2.waitKey(0)
cv2.destroyAllWindows()

(460, 307, 3)
(368, 246, 3)


In [16]:
cv2.imwrite("sample1.jpg",res)

True

# add noise

In [5]:
def add_gaussian_noise(image, sigma):
    '''Add Gaussian noise to an image.
       image : gray level image, pixel intensity : 0 ~255
       simga : float, standard deviation, controls level of noise 
       mu(mean) = 0.0 , float
    '''
    mu = 0.0 
    # Draw random samples from a normal (Gaussian) distribution
    noise = np.random.normal(mu, sigma, (image.shape))
    # generate noisy image
    noise_image = image + noise
    # clip the values to the interval 0 ~ 255
    noise_image = np.clip( noise_image, 0, 255)
    # convert 'float' to 'uint8'
    noise_image = noise_image.astype('uint8')
    return noise_image

In [6]:
def add_salt_and_pepper(image, threshold):
    '''Add " Salt & Pepper " noise to an image.
       image : gray-level image, pixel intensity : 0 ~ 255
       threshold : probability that controls level of noise
    '''
    # np.random.rand : create random values in a given shape
    # random samples from uniform distribution over [0,1)
    # rnd : ndarray, random values
    rnd = np.random.rand(image.shape[0], image.shape[1], image.shape[2])
    noise_image = image.copy()
    noise_image[ rnd < threshold] = 0
    noise_image[ rnd > (1- threshold)] = 255
    return noise_image

In [126]:
# add gaussian noise to an image with simga(standard deviation) = 10
imagepath = "sample2.jpg"
img = cv2.imread(imagepath)
noise_img = add_gaussian_noise(img, 10)

In [127]:
noise_img2 = add_gaussian_noise(noise_img, 20)

In [128]:
cv2.imshow('Noise Image', noise_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [129]:
cv2.imwrite('smaple2_gauss10_20.jpg',noise_img2)

True

In [134]:
# add salt-and-pepper noise to an image with threshold = 0.01
imagepath = "sample3.jpg"
img = cv2.imread(imagepath)
noise_img = add_salt_and_pepper(img, 0.01)

In [135]:
noise_img2 = add_salt_and_pepper(noise_img , 0.03)

In [136]:
cv2.imshow('Noise Image', noise_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [137]:
cv2.imwrite('smaple3_sp01_03.jpg',noise_img2)

True

# image enhancement

In [10]:
def modified_unshark_masking(img, smooth_img,c):
    ''' 
    img : gray level image
    c: constant, 3/5(0.6) ~ 5/6(0.833)
    img_avg : image after using 3x3 average filter 
    if ( c < 3/5 or c > 5/6 ):
        print('Use constant c between 3/5 and 5/6')
        return -1
        
    '''
    #output_image = (c/(2*c-1)) * img - ((1-c)/(2*c-1)) * smooth_img
    output_image = c* img - (1-c) * smooth_img
    print(output_image.dtype)
    output_image[ output_image < 0] = 0
    output_image[ output_image > 255 ] = 255
    return  output_image.astype(np.uint8)

In [15]:
"""
pascale.jpg --- enhancement
stacey.jpg --- noise
jason.jpg --- noise
michael.jpg --- detail 3 iteration
ricardo.jpg --- detail 3 iteration
daniel.jpg --- enhancement
"""

imagepath ="sample6.jpg"
kernelsize = 3
iteration =1

I = cv2.imread(imagepath)
M= I.copy()
for i in range(iteration):
    print('Iteration: ', str(i+1))
    M,localmax, localmin, maxenvelope, minenvelope = EdgePreservingSmooth(M,kernelsize)
    kernelsize += 4
    print('')

I_YUV = cv2.cvtColor(I,cv2.COLOR_BGR2YUV)
M_YUV = cv2.cvtColor(M,cv2.COLOR_BGR2YUV)
D = I_YUV[:,:,0]-M_YUV[:,:,0]

# Make the grey scale image have three channels
grey_3_channel = cv2.cvtColor(D, cv2.COLOR_GRAY2BGR)
numpy_horizontal = np.hstack(( I, M, grey_3_channel))
cv2.imshow('Edge-preserving Smooth Result', numpy_horizontal)
cv2.waitKey()
cv2.destroyAllWindows()

Iteration:  1
Identifcation of local minima and local maxima of I
Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively
Computation of the smoothed mean M as the average of the extremal envelopes



In [16]:
cv2.imwrite("sample6_smooth.jpg",numpy_horizontal)

True

In [21]:
enhance_img1 = modified_unshark_masking(I, M,0.9)
enhance_img2 = modified_unshark_masking(I, M,1.1)
numpy_horizontal = np.hstack(( I,M,enhance_img1, enhance_img2))
cv2.imshow('Enhancement', numpy_horizontal)
cv2.waitKey()
cv2.destroyAllWindows()

float64
float64


In [22]:
cv2.imwrite("sample6_enhancement09_11.jpg", numpy_horizontal)

True

# remove noise with 2 iterations

In [32]:
"""
pascale.jpg --- enhancement
stacey.jpg --- noise
jason.jpg --- noise
michael.jpg --- detail 3 iteration
ricardo.jpg --- detail 3 iteration
daniel.jpg --- enhancement
"""
"""
smaple2_gauss10_20.jpg
smaple2_sp01_03.jpg
smaple3_gauss10_20.jpg
smaple3_sp01_03.jpg
"""

imagepath ="smaple3_sp01_03.jpg"
kernelsize = 3
iteration =2

Mlist = []

I = cv2.imread(imagepath)
M= I.copy()
for i in range(iteration):
    print('Iteration: ', str(i+1))
    M,localmax, localmin, maxenvelope, minenvelope = EdgePreservingSmooth(M,kernelsize)
    Mlist.append(M)
    kernelsize += 4
    print('')

I_YUV = cv2.cvtColor(I,cv2.COLOR_BGR2YUV)
M_YUV = cv2.cvtColor(M,cv2.COLOR_BGR2YUV)
D = I_YUV[:,:,0]-M_YUV[:,:,0]

# Make the grey scale image have three channels
grey_3_channel = cv2.cvtColor(D, cv2.COLOR_GRAY2BGR)
numpy_horizontal = np.hstack(( I, M, grey_3_channel))
cv2.imshow('Edge-preserving Smooth Result', numpy_horizontal)
cv2.waitKey()
cv2.destroyAllWindows()

Iteration:  1
Identifcation of local minima and local maxima of I
Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively
Computation of the smoothed mean M as the average of the extremal envelopes

Iteration:  2
Identifcation of local minima and local maxima of I
Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively
Computation of the smoothed mean M as the average of the extremal envelopes



In [33]:
# (M*255).astype(np.uint8)
# normalize(Mlist[0]) normalize(Mlist[1])
numpy_horizontal = np.hstack(( I,Mlist[0], Mlist[1]))
cv2.imshow('Noise Removal', numpy_horizontal)
cv2.waitKey()
cv2.destroyAllWindows()

In [34]:
cv2.imwrite("sample3_sp_remove.jpg", numpy_horizontal)

True

# 3 iterations with details

In [7]:
"""
pascale.jpg --- enhancement
stacey.jpg --- noise
jason.jpg --- noise
michael.jpg --- detail 3 iteration
ricardo.jpg --- detail 3 iteration
daniel.jpg --- enhancement
"""


imagepath ="sample5.jpg"
kernelsize = 3
iteration =3

Mlist = []
Dlist = []

I = cv2.imread(imagepath)
M= I.copy()
for i in range(iteration):
    print('Iteration: ', str(i+1))
    M,localmax, localmin, maxenvelope, minenvelope = EdgePreservingSmooth(M,kernelsize)
    Mlist.append(M)
    
    I_YUV = cv2.cvtColor(I,cv2.COLOR_BGR2YUV)
    M_YUV = cv2.cvtColor(M,cv2.COLOR_BGR2YUV)
    D = I_YUV[:,:,0]-M_YUV[:,:,0]
    # Make the grey scale image have three channels
    grey_3_channel = cv2.cvtColor(D, cv2.COLOR_GRAY2BGR)
    Dlist.append(grey_3_channel)
    
    kernelsize += 4
    print('')

#I_YUV = cv2.cvtColor(I,cv2.COLOR_BGR2YUV)
#M_YUV = cv2.cvtColor(normalize(M),cv2.COLOR_BGR2YUV)
D = I_YUV[:,:,0]-M_YUV[:,:,0]

# Make the grey scale image have three channels
#grey_3_channel = cv2.cvtColor(D, cv2.COLOR_GRAY2BGR)
numpy_horizontal = np.hstack(( I, M, Dlist[-1]))
cv2.imshow('Edge-preserving Smooth Result', numpy_horizontal)
cv2.waitKey()
cv2.destroyAllWindows()

Iteration:  1
Identifcation of local minima and local maxima of I
Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively
Computation of the smoothed mean M as the average of the extremal envelopes

Iteration:  2
Identifcation of local minima and local maxima of I
Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively
Computation of the smoothed mean M as the average of the extremal envelopes

Iteration:  3
Identifcation of local minima and local maxima of I
Interpolation of the local minima and maxima to compute minimal and maximal extremal envelopes respectively
Computation of the smoothed mean M as the average of the extremal envelopes



In [8]:
# (M*255).astype(np.uint8)
#  normalize(Mlist[0]) normalize(Mlist[1]) normalize(Mlist[2])
Zero = np.zeros(I.shape, dtype=I.dtype)
numpy_horizontal1 = np.hstack(( I,Mlist[0], Mlist[1],Mlist[2]))
numpy_horizontal2 = np.hstack((Zero,Dlist[0],Dlist[1],Dlist[2]))
result = np.vstack((numpy_horizontal1, numpy_horizontal2))
cv2.imshow('3 Iterations', result)
cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
cv2.imwrite("sample5_3iterations.jpg", result)

True